In [2]:
import bioframe as bf
import anndata
import pandas as pd
import os
import sys

sys.path.append('/code/decima/src/decima')
from variant import process_variants

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Paths

In [3]:
matrix_file='/gstore/data/resbioai/grelu/decima/20240823/data.h5ad'
h5_file='/gstore/data/resbioai/grelu/decima/20240823/data.h5'

out_dir='/gstore/data/resbioai/grelu/decima/20240823/gwas_44traits/positive_variants'
variant_file = os.path.join(out_dir, 'positive_variants.csv')

## Load data

In [9]:
ad = anndata.read_h5ad(matrix_file)
variants = pd.read_csv(variant_file)

## Process variants

In [ ]:
variants = process_variants(variants, ad)

## Save

In [ ]:
var_out_file = os.path.join(out_dir, 'positive_variants_processed.csv')
variants.to_csv(var_out_file, index=None)

## Run Decima

In [11]:
device = 0
out_file = os.path.join(out_dir, 'decima_preds.npy')
ckpts = ['/gstore/data/resbioai/grelu/decima/20240823/lightning_logs/kugrjb50/checkpoints/epoch=3-step=2920.ckpt',
        '/gstore/data/resbioai/grelu/decima/20240823/lightning_logs/0as9e8of/checkpoints/epoch=7-step=5840.ckpt',
        '/gstore/data/resbioai/grelu/decima/20240823/lightning_logs/i68hdsdk/checkpoints/epoch=2-step=2190.ckpt',
        '/gstore/data/resbioai/grelu/decima/20240823/lightning_logs/i9zsp4nm/checkpoints/epoch=8-step=6570.ckpt'
        ]

cmd = f"CUDA_VISIBLE_DEVICES={device} python /code/decima/scripts/vep.py \
-device {device} -ckpts {' '.join(ckpts)} \
-h5_file {h5_file} -variant_df_file {var_out_file} \
-out_file {out_file}"
print(cmd)

CUDA_VISIBLE_DEVICES=0 python /code/decima/scripts/vep.py -device 0 -ckpts /gstore/data/resbioai/grelu/decima/20240823/lightning_logs/kugrjb50/checkpoints/epoch=3-step=2920.ckpt /gstore/data/resbioai/grelu/decima/20240823/lightning_logs/0as9e8of/checkpoints/epoch=7-step=5840.ckpt /gstore/data/resbioai/grelu/decima/20240823/lightning_logs/i68hdsdk/checkpoints/epoch=2-step=2190.ckpt /gstore/data/resbioai/grelu/decima/20240823/lightning_logs/i9zsp4nm/checkpoints/epoch=8-step=6570.ckpt -h5_file /gstore/data/resbioai/grelu/decima/20240823/data.h5 -variant_df_file /gstore/data/resbioai/grelu/decima/20240823/gwas_44traits/positive_variants/positive_variants_processed.csv -out_file /gstore/data/resbioai/grelu/decima/20240823/gwas_44traits/positive_variants/decima_preds.npy


## Run Borzoi

In [14]:
device = 1
cmd = f"CUDA_VISIBLE_DEVICES={device} python /code/decima/scripts/vep_borzoi.py \
-device {device} -h5_file {h5_file} -variant_df_file {var_out_file} \
-out_dir {out_dir}"
print(cmd)

CUDA_VISIBLE_DEVICES=1 python /code/decima/scripts/vep_borzoi.py -device 1 -h5_file /gstore/data/resbioai/grelu/decima/20240823/data.h5 -variant_df_file /gstore/data/resbioai/grelu/decima/20240823/gwas_44traits/positive_variants/positive_variants_processed.csv -out_dir /gstore/data/resbioai/grelu/decima/20240823/gwas_44traits/positive_variants
